In [1]:
#%matplotlib inline
import numpy as np
#from pylab import *
#from sympy import *
import sympy as sym

In [2]:
def linfty_norm(fun,a=0,b=1,nn=20):
    xx = np.linspace(a,b,2**nn+1)
    return np.amax(np.absolute(fun(xx)))

def lagrangian_interpolation_matrix(x,q):
    n = q.shape[0]-1
    Ln = np.zeros((n+1, len(x)))

    for i in xrange(n+1):
        Ln[i] = np.product([ (x-q[j])/(q[i]-q[j]) for j in xrange(n+1) if j != i], axis=0)
        
    return Ln

def lagrangian_interpolation(f,q):
    return lambda x : np.dot(np.transpose(lagrangian_interpolation_matrix(x,q)),f(q))
    

In [18]:
x = sym.symbols('x')

f = sym.sin(2.*sym.pi*x)

df = sym.diff(f,x,3)

print(df)

nf = sym.lambdify(x,f)

print nf(np.array([1]))

print(f)

-8.0*pi**3*cos(2.0*pi*x)
-2.44929359829e-16
sin(2.0*pi*x)


In [ ]:
ref = 2**10+1 # So that x_i+1 - x_i is exactly representable in base 2
x = np.linspace(0,1,ref)

deg = 5

f = lambda x: np.sin(2*np.pi*x)

q = np.linspace(0,1,deg+1)

l = lagrangian_interpolation(f,q)

linfty = linfty_norm(lambda x : f(x)-l(x))

In [ ]:




figure(figsize=[10,5])



for n in xrange(2,20):
    # The interpolation points
    q = linspace(0,1,n+1)
    

    # The interpolation function
    f = lambda x: sin(2*pi*x)
    
    #Ln = lagrangian_interpolation_matrix(x,q)
    l = lagrangian_interpolation(f,q)
    
    linfty=amax(absolute(l(x)-f(x)))
    # t = symbols('t')
    #deri=diff(lambda x: sin(2*pi*x), 1.0,2)
    
    print linfty
    #figure(figsize=[10,5])
    _ = semilogy (n,linfty,'ro')
    

figure(figsize=[10,5])
_ = plot(x, f(x))
_ = plot(x, l(x))
_ = plot(q, f(q), 'ro')